In [2]:
import xarray as xr
import pandas as pd
import numpy as np

In [3]:
#load data with xarray
file_path = r"d:\klein\datasets_DEcoImpact\conv_schism_raw_2.nc"
xr_file = xr.open_dataset(file_path)
print(xr_file)

<xarray.Dataset> Size: 6GB
Dimensions:                  (nSCHISM_hgrid_face: 1019883,
                              nMaxSCHISM_hgrid_face_nodes: 4,
                              nSCHISM_hgrid_node: 515582, time: 24,
                              nSCHISM_vgrid_layers: 21)
Coordinates:
  * time                     (time) datetime64[ns] 192B 2017-01-03T01:00:00 ....
Dimensions without coordinates: nSCHISM_hgrid_face,
                                nMaxSCHISM_hgrid_face_nodes,
                                nSCHISM_hgrid_node, nSCHISM_vgrid_layers
Data variables: (12/14)
    SCHISM_hgrid_face_nodes  (nSCHISM_hgrid_face, nMaxSCHISM_hgrid_face_nodes) int32 16MB ...
    SCHISM_hgrid_node_x      (nSCHISM_hgrid_node) float64 4MB ...
    SCHISM_hgrid_node_y      (nSCHISM_hgrid_node) float64 4MB ...
    depth                    (nSCHISM_hgrid_node) float32 2MB ...
    dryFlagElement           (time, nSCHISM_hgrid_face) float32 98MB ...
    dryFlagNode              (time, nSCHISM_hgrid_node) flo

In [4]:
#Subset on time to reduce size
sub_file = xr_file.sel(time = slice("2017-01-03 00:00:00", "2017-01-03 04:00:00"))
time_attrs = sub_file["time"].attrs
#time_attrs["_FillValue"] = False
sub_file["time"] = sub_file["time"].assign_attrs(time_attrs)


In [5]:
#Subset on faces to reduce size
faces_needed = [
    564616, 564617, 566049, 566050, 566051, 566052, 566053, 566054, 566055, 566056, 566057, 566058, 566059, 566060, 
    566064, 567521, 567522, 567523, 567524, 567525, 567526, 567527, 567528, 567529, 567530, 567531, 567532, 567533, 
    567534, 567535, 567536, 567537, 567538, 567539, 567540, 567548, 567549, 569007, 569008, 569009, 569010, 569011, 
    569012, 569013, 569014, 569015, 569016, 569017, 569018, 569019, 569020, 569021, 569022, 569023, 569031, 569033, 
    569034, 569035, 569036, 570500, 570501, 570502, 570503, 570504, 570505, 570506, 570507, 570508, 570509, 570510, 
    570511, 570512, 570513, 570514, 570515, 570516, 570517, 570518, 570519, 570520, 570521, 570522, 570523, 570524, 
    570525, 570526, 570527, 570528, 570529, 570530, 570541, 570542, 570543, 571998, 571999, 572000, 572001, 572002, 
    572003, 572004, 572005, 572006, 572007, 572008, 572009, 572010, 572011, 572012, 572013, 572014, 572015, 572016, 
    572017, 572018, 572019, 572020, 572021, 572022, 572023, 572024, 572025, 572026, 572027, 572028, 572029, 572030, 
    572031, 572042, 572044, 572045, 573526, 573527, 573528, 573529, 573530, 573531, 573532, 573533, 573534, 573535, 
    573536, 573537, 573538, 573539, 573540, 573541, 573542, 573543, 573544, 573545, 573546, 573547, 573548, 573549, 
    573550, 573551, 573552, 573553, 573554, 573555, 573556, 573557, 573558, 573571, 575079, 575080, 575081, 575082, 
    575083, 575084, 575085, 575086, 575087, 575088, 575089, 575090, 575091, 575092, 575093, 575094, 575095, 575096, 
    575097, 575098, 575099, 575100, 575101, 575102, 575103, 575104, 575105, 575106, 575107, 575108, 575109, 575120, 
    576631, 576632, 576633, 576634, 576635, 576636, 576637, 576638, 576639, 576640, 576641, 576642, 576643, 576644, 
    576645, 576646, 576647, 576648, 576649, 576650, 576651, 576652, 576653, 576654, 576655, 576656, 576657, 576669, 
    578167, 578168, 578169, 578170, 578171, 578172, 578173, 578174, 578175, 578176, 578177, 578178, 578179, 578180, 
    578181, 578182, 578183, 578184, 578185, 578186, 578187, 578188, 578202, 579714, 579715, 579716, 579717, 579718, 
    579719, 579720, 579721, 579722, 579723, 579724, 579725, 579726, 579727, 579728, 579729, 579730, 579731, 579732, 
    579747, 581259, 581260, 581261, 581262, 581263, 581264, 581265, 581266, 581267, 581268, 581269, 581270, 581271, 
    581272, 581273, 581274, 581289, 582793, 582794, 582795, 582796, 582797, 582798, 582799, 582800, 582801, 582802, 
    582803, 582804, 582805, 582806, 582807, 584323, 584324, 584325, 584326, 584327, 584328, 584329, 584330, 584331, 
    584332, 584333, 584334, 584335, 585860, 585861, 585862, 585863, 585864, 585865, 585866, 585867, 585868, 585869, 
    585870, 587399, 587400, 587401, 587402, 587403, 587404, 587405, 587406, 587407, 588943, 588944, 588945, 588946, 
    588947, 588949,  
    ]

#create dataframe for selection
face_nodes_dataframe = pd.DataFrame(sub_file.SCHISM_hgrid_face_nodes.data)
face_nodes_dataframe = face_nodes_dataframe.replace(-1,np.NaN)

#retrieve nodes and faces of importance from dataframe
face_nodes_needed_dataframe = face_nodes_dataframe.iloc[faces_needed]

#filter on nodes from dataset data
sub_file = sub_file.sel(nSCHISM_hgrid_node = xr_file.nSCHISM_hgrid_node.isin(face_nodes_needed_dataframe), drop = True)

#filter on faces from dataset data
sub_file = sub_file.sel(nSCHISM_hgrid_face = sub_file.nSCHISM_hgrid_face.isin(faces_needed))

#renumber nodes in face_nodes table
list_nodes = face_nodes_needed_dataframe.to_numpy().flatten().tolist()
list_nodes = list(np.array(list_nodes)[~np.isnan(np.array(list_nodes))])
list_nodes = list(set(sorted(list_nodes)))

ar = sub_file.SCHISM_hgrid_face_nodes.data
k = np.array(list_nodes) 
v = np.array(range(0,len(list_nodes)))

#replace
copy_data = sub_file.SCHISM_hgrid_face_nodes.data.copy()
for key, value in zip(k,v):
    copy_data[sub_file.SCHISM_hgrid_face_nodes.data == key] = value

#reset to non-existing cells to -1
#copy_data[np.isnan(copy_data)] = -1

#replace with the corrected data
sub_file.SCHISM_hgrid_face_nodes.data = copy_data

#set attributes for -1 fillvalue
#face_nodes_attrs = sub_file["SCHISM_hgrid_face_nodes"].attrs
#face_nodes_attrs["_FillValue"] = False
#sub_file["SCHISM_hgrid_face_nodes"] = sub_file["SCHISM_hgrid_face_nodes"].assign_attrs(time_attrs)

#Check
print(sub_file)
#print(xr_file.SCHISM_hgrid_node_y.sel(nSCHISM_hgrid_node = xr_file.nSCHISM_hgrid_node.isin(face_nodes_needed_dataframe)).values[-20:-1])
#print(sub_file.SCHISM_hgrid_node_y.values[-20:-1])

<xarray.Dataset> Size: 867kB
Dimensions:                  (nSCHISM_hgrid_face: 338,
                              nMaxSCHISM_hgrid_face_nodes: 4,
                              nSCHISM_hgrid_node: 414, time: 4,
                              nSCHISM_vgrid_layers: 21)
Coordinates:
  * time                     (time) datetime64[ns] 32B 2017-01-03T01:00:00 .....
Dimensions without coordinates: nSCHISM_hgrid_face,
                                nMaxSCHISM_hgrid_face_nodes,
                                nSCHISM_hgrid_node, nSCHISM_vgrid_layers
Data variables: (12/14)
    SCHISM_hgrid_face_nodes  (nSCHISM_hgrid_face, nMaxSCHISM_hgrid_face_nodes) int32 5kB ...
    SCHISM_hgrid_node_x      (nSCHISM_hgrid_node) float64 3kB ...
    SCHISM_hgrid_node_y      (nSCHISM_hgrid_node) float64 3kB ...
    depth                    (nSCHISM_hgrid_node) float32 2kB ...
    dryFlagElement           (time, nSCHISM_hgrid_face) float32 5kB ...
    dryFlagNode              (time, nSCHISM_hgrid_node) float32 7kB

In [6]:
#write to file example
sub_file.to_netcdf("rawdata/schism_output_example.nc")